In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pulp
import itertools
import os
import pickle

from tqdm import tqdm

df = pd.read_csv("dataset/finished_dataset.csv")
df_filter = pd.read_csv("requests_done_simulation.csv")
filter_ids = df_filter["id"].tolist()
df = df[df["id"].isin(filter_ids)]
with open('graph_from_routes_wd.pkl', 'rb') as f:
    graph_from_routes = pickle.load(f)



In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_date,location_pickup,location_dropoff,pickup_graph_node,dropoff_graph_node
286,286,5477,id2419964,2,2016-03-14 09:43:07,2016-03-14 09:58:46,1,-73.963844,40.768314,-73.983986,40.767921,N,939,2016-03-14,21,59,42437654,6223571524
310,310,5930,id2067847,2,2016-03-23 23:31:59,2016-03-23 23:41:04,1,-73.975281,40.752838,-73.997299,40.752079,N,545,2016-03-23,12,29,42430828,6177439750
379,379,7777,id2433768,1,2016-03-18 18:48:55,2016-03-18 19:17:30,1,-73.968277,40.787018,-73.993408,40.754009,N,1715,2016-03-18,10,29,42440743,6177439750
415,415,8498,id2792744,2,2016-03-21 21:35:23,2016-03-21 21:51:25,1,-73.974297,40.753738,-73.971741,40.753731,N,962,2016-03-21,12,1,42430828,42451593
1323,1323,27899,id1224697,1,2016-03-15 12:33:45,2016-03-15 12:42:16,1,-73.991882,40.719772,-74.005653,40.716587,N,511,2016-03-15,27,15,42447020,42436439


In [4]:
nyc_graph = graph_from_routes.copy()

In [5]:
import math
edge_info = {}
l = 50 #km/h#/ 3600#m/h
# Iterate over edges and add distances to the dictionary
for u, v, d in nyc_graph.edges(data=True):
    edge_info[(u, v)] = [
        d["length"],
        d["length"] / d["maxspeed"] if not math.isnan(d["maxspeed"]) else d["length"]/l,
        50
    ]

In [6]:
edge_info

{(42433027, 42433020): [173.67300000000003, 6.946920000000001, 50],
 (42433020, 42433027): [173.67300000000003, 6.946920000000001, 50],
 (42433020, 42433014): [132.314, 2.64628, 50],
 (42433014, 42433020): [132.314, 2.64628, 50],
 (42433014, 42442671): [57.784, 1.15568, 50],
 (42442671, 42433014): [57.784, 1.15568, 50],
 (42442671, 42442664): [80.288, 1.6057599999999999, 50],
 (42442671, 42432999): [560.6172082192738, 11.212344164385478, 50],
 (42442664, 42442671): [80.288, 1.6057599999999999, 50],
 (42442664, 42432999): [57.055, 1.1411, 50],
 (42442664, 42442651): [401.383, 8.02766, 50],
 (42432999, 42442664): [57.055, 1.1411, 50],
 (42432999, 42432985): [415.863, 16.63452, 50],
 (42432999, 42442671): [560.6172082192738, 11.212344164385478, 50],
 (42432985, 42432999): [415.863, 16.63452, 50],
 (42432985, 42432963): [817.7499999999999, 32.709999999999994, 50],
 (42432985, 42442651): [155.88299999999998, 3.1176599999999994, 50],
 (42432963, 42432985): [817.7499999999999, 32.709999999999

In [5]:
edge_info = {}
l = 50 #km/h#/ 3600#m/h
# Iterate over edges and add distances to the dictionary
for u, v, d in nyc_graph.edges(data=True):
    edge_info[(u, v)] = [
        d["length"]/10,
        d["length"]/500,
        50
    ]

In [6]:
from enum import Enum
class Charger(Enum):
    SLOW = (0.1,00.6)
    MEDIUM = (0.1,00.8)
    FAST = (0.2,00.8)

In [7]:


class Depot():
    def __init__(self, node_id, type = Charger.SLOW, vcapacity = 50, vehicles_now = 25) -> None:
        self.id = node_id
        self.type = type
        self.r, _ = type.value
        self.vcapacity_ = vcapacity
        self.vehicles_now = vehicles_now


In [8]:
class Vehicle():
    def __init__(self, id, pcapacity=4,gcapacity = 10, 
                 R_ = 1.5, 
                 ctype = Charger.SLOW,
                 start_depot = 0, 
                 end_depot = 0, 
                 Q = 100,
                 tau = None,
                 ):
        self.id = id
        self.pcapacity_ = pcapacity
        self.gcapacity_ = gcapacity
        self.R_ = R_
        self.charge = 100
        self.start_depot = start_depot
        self.end_depot = end_depot
        self.requests_ ={}
        _,self.theta = ctype.value
        self.Q = Q
        self.tau = tau if tau is not None else 20/(self.R_*self.theta)
    def update_requests(self, id, start, dest):
        self.requests_[id] = (start,dest)
    def update_requests(self, request):
        self.requests_[request.id] = (request.start,request.dest)
    def update_charge(self,distance, speed = 50):
        self.charge -=  distance/speed * self.R_


In [9]:
class Request():
    def __init__(self, id, people, start, end,  py=0,px=0, dy=0,dx=0) -> None:
        self.id = id
        self.people_ = people
        self.start = start
        self.end = end 
        self.px = px
        self.py=py
        self.dx = dx
        self.dy = dy


In [10]:
def convert_reqs_to_obj(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]):
    reqs_obj = {}
    for node, item in requests.items():
        amount = item["amount"]
        temp_reqs = []
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
        reqs_obj[node] = temp_reqs
    
    return reqs_obj

In [11]:
def convert_reqs_to_list(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]):
    temp_reqs = []
    for node, item in requests.items():
        amount = item["amount"]
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
    return temp_reqs

In [12]:
def get_requests(dataframe, required_info = ["amount","internal_ids", "id", "passenger_count", "pickup_graph_node","dropoff_graph_node","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]):
    requests = {}
    pick_up_nodes = np.unique(dataframe["pickup_graph_node"])
    
    for node in pick_up_nodes:
        temp_dict = dataframe[dataframe["pickup_graph_node"]==node].to_dict()
        temp_dict["amount"] = len(temp_dict["id"])
        temp_dict["internal_ids"] = list(temp_dict["id"].keys())
        requests[node] = {x:temp_dict[x] for x in required_info}
    reqs_obj = convert_reqs_to_obj(requests)
    reqs_list = convert_reqs_to_list(requests)
    return requests, reqs_obj, reqs_list

In [13]:
def initialize_depots(graph, vehicles_amount = [20]*5,type = [Charger.FAST]*5 ):
    depots = []
    idx = 0
    for node_id, data in graph.nodes(data = True):
        if data["depot"]:
            depots += [Depot(node_id, type = type[idx], vehicles_now = vehicles_amount[idx])]
            idx += 1
    return depots

In [14]:
def initialize_vehicles_naive(depots):
    vehicles = []
    # Calculate the total number of vehicles at the beginning
    n_vehicles = sum(d.vehicles_now for d in depots)
    vehicle_id_counter = 0  # Start assigning IDs from 1
    for i, d in enumerate(depots):
        start_depot_id = d.id
        end_depot_id =depots[(i + 1) % len(depots)].id  # Ensures start and end depots are different
        for _ in range(d.vehicles_now):
            vehicles.append(Vehicle(id=vehicle_id_counter,
                                    pcapacity=5,
                                    gcapacity=10,
                                    R_=1,
                                    ctype = d.type,
                                    start_depot=start_depot_id,
                                    end_depot=end_depot_id))
            vehicle_id_counter += 1  # Increment the ID counter
    return vehicles, n_vehicles


In [15]:
import random
def initialize_vehicles_random(depots):
    vehicles = []
    n_vehicles = sum(d.vehicles_now for d in depots)
    vehicle_id_counter = 0
    for i, d in enumerate(depots):
        start_depot_id = d.id
        # Choose a random end depot that is different from the start depot
        end_depot = random.choice([depot for depot in depots if depot.id != start_depot_id])
        end_depot_id = end_depot.id
        for _ in range(d.vehicles_now):
            vehicles.append(Vehicle(id=vehicle_id_counter,
                                    pcapacity=5,
                                    gcapacity=10,
                                    R_=1,
                                    ctype = d.type,
                                    start_depot=start_depot_id,
                                    end_depot=end_depot_id))
            vehicle_id_counter += 1
    return vehicles, n_vehicles

In [16]:
def initialize_vehicles_with_specific_start_end(depots, end_depots = {42432963:42430044,42430044: 6177439750, 6223571524:42432963, 3786901738:42432963, 6177439750:6223571524}):
    vehicles = []
    n_vehicles = sum(d.vehicles_now for d in depots)
    vehicle_id_counter = 0
    for i, d in enumerate(depots):
        start_depot_id = d.id
        end_depot_id = end_depots[start_depot_id]
        for _ in range(d.vehicles_now):
            vehicles.append(Vehicle(id=vehicle_id_counter,
                                    pcapacity=2,
                                    gcapacity=10,
                                    R_=1,
                                    ctype = d.type,
                                    start_depot=start_depot_id,
                                    end_depot=end_depot_id))
            vehicle_id_counter += 1
    return vehicles, n_vehicles


In [17]:
depots = initialize_depots(nyc_graph, vehicles_amount = [1]*5)
#vehicles, _ = initialize_vehicles_naive(depots)

In [18]:
depot_ids = [d.id for d in depots]  # Generate the list of depot ids

# Replace values with NaN where dropoff_graph_node is in the list of depot ids
df.loc[df['dropoff_graph_node'].isin(depot_ids), 'dropoff_graph_node'] = np.nan
mode_value = df['dropoff_graph_node'].mode()[0]
df['dropoff_graph_node'].fillna(mode_value, inplace=True)
df.loc[df['pickup_graph_node'].isin(depot_ids), 'pickup_graph_node'] = np.nan
mode_value = df['pickup_graph_node'].mode()[0]
df['pickup_graph_node'].fillna(mode_value, inplace=True)

In [19]:
def clean_requests(requests, depots, road_network):

    tot = 0
    for r in requests:
        #if r.people_ >4:
        #    requests.remove(r)
        #    continue
            
    
        #for d in depots:
        #    if r.end == d.id or r.start == d.id:
        #        requests.remove(r)
        #        continue
                

        if (r.start, r.end) not in road_network.edges():
            requests.remove(r)
            continue
            
        
        tot = r.people_ + tot

    return requests, tot

In [20]:
edges_to_remove = [(u, v) for u, v in nyc_graph.edges() if u == v]
nyc_graph.remove_edges_from(edges_to_remove)

In [21]:
def check_road(graph, id, depot_id, V,s):
    raw_road = {}
    times = {}
    arriving_times = {}
    lrr = 0
    for u, v in graph.edges():
        if pulp.value(V[id,u, v]) >0.5:
            raw_road[u] = v 
            if s != None:
                times[u] = pulp.value(s[id,u])
                arriving_times[v] = pulp.value(s[id,v])
            lrr+=1
    n = depot_id
    final_road = {}
    if s != None:
        print(times)
        print(arriving_times)
    print(lrr,raw_road)
    print( depot_id)
    for i in range(lrr):
        print(i+1,n)
        final_road[n] = raw_road[n]
        n = raw_road[n]

    print(len(final_road.keys()))

    print(final_road)
    return final_road
        

In [22]:
def choose_requests(requests,graph, amount, from_index):
    picked = []
    idx = from_index
    while len(picked) != amount:
        r = requests[idx]
        idx+=1
        if idx > len(requests):
            return -1
        if r.end not in graph.nodes():
            print("r.end not in nodes")
            continue
        if r.start not in graph.nodes():
            print("Req not in nodes")
            continue
        if r.start not in graph.nodes():
            print("Req not in nodes")
            continue
        picked += [r]

    return picked

In [23]:
def update_vehicles(vehicles, graph, V, edges_info):
    for a in vehicles:
        new_charge = a.charge
        for u,v in graph.edges():
            try:
                new_charge -= int(pulp.value(V[a.id,u,v]) > 0.5)*a.R_*edges_info[u,v][1]  
            except:
                pass
        if new_charge != a.charge:
                old = a.end_depot 
                a.end_depot = a.start_depot
                a.start_depot = old
                a.charge = new_charge
        
        #print(f"Vehicle {a.id} soc {a.charge}")
            #a.charge += timing_charge
    

In [24]:
def update_depots(depots, vehicles, timing_charge = "fill"):
    #for d in depots:
    #    print(f"{d.id} # vehicles: {d.vehicles_now}")
    #print()
    for d in depots:
        vehicles_now = 0
        for a in vehicles:
            if a.start_depot == d.id:
                vehicles_now +=1
                if timing_charge == "fill":
                    new_charge = a.Q
                else:
                    b = a.Q*a.theta / a.R_
                    if timing_charge <= b:
                        new_charge = a.charge + a.R_* timing_charge
                    if timing_charge > b: 
                        new_charge = a.charge + a.R_*b
                        i_t = a.R_*np.exp(-(timing_charge-b)/a.tau)
                        new_charge+= a.Q - a.Q*i_t*(1-a.theta)/a.R_

                a.charge = new_charge if new_charge< 100 else 100
        d.vehicles_now = vehicles_now
    #    print(f"{d.id} # vehicles: {d.vehicles_now}")
    
    #for a in vehicles:
        #print(f"Vehicle {a.id} soc: {a.charge}")
        #print("after:", d.vehicles_now)
        

In [25]:

def check_and_create_folder(folder_path, name_fold = "simu"):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        #return folder_path

    # Look for folders starting with "simu_"
    existing_folders = [name for name in os.listdir(folder_path) if name.startswith(f"{name_fold}_")]
    #print(f"{name}_")
    if not existing_folders:
        new_folder_name = f"{name_fold}_0001"
    else:
        # Sort the existing folders to find the highest numbered folder
        existing_folders.sort()
        last_folder_number = int(existing_folders[-1][len(name_fold)+1:])  # Extract the number from the folder name 
        #print(os.path.join(folder_path, existing_folders[-1]), len(os.listdir(os.path.join(folder_path, existing_folders[-1]))))   
        new_folder_number = last_folder_number  if  len(os.listdir(os.path.join(folder_path, existing_folders[-1]))) ==0 else last_folder_number +1
        #print(new_folder_number)
        new_folder_name = f"{name_fold}_{new_folder_number:04d}"

    new_folder_path = os.path.join(folder_path, new_folder_name)
    os.makedirs(new_folder_path, exist_ok=True)

    return new_folder_path

In [26]:
def insert_random_end_point(requests, graph):
    for r in requests:
        r.end = random.choice([x for x in graph.nodes() if x != r.start])
    return requests
    

In [27]:
p, _, requests = get_requests(df[:300])

depots = initialize_depots(nyc_graph, vehicles_amount = [4]*5)
#requests, tot = clean_requests(requests, depots, graph)
random.shuffle(requests)
vehicles, _ = initialize_vehicles_with_specific_start_end(depots)


In [28]:
len(nyc_graph.edges)

1622

In [29]:
for r in requests:
    if (r.start,r.end) not in nyc_graph.edges():
        nyc_graph.add_edge(r.start,r.end)
    if (r.end,r.start) not in nyc_graph.edges():
        nyc_graph.add_edge(r.end,r.start)

In [30]:
len(nyc_graph.edges())

1763

In [31]:
tot = 0
for r in requests:
    if r.people_ %2 !=0:
        r.people_+=1
        #print(r.id, r.people_)
    tot +=r.people_
#print(tot)

In [32]:
test_vehicles, _ = initialize_vehicles_with_specific_start_end(depots)

In [33]:
test_g = nx.DiGraph()

# Add nodes
test_g.add_nodes_from(range(5,20))

# Add edges between all pairs of nodes
for i in range(5,20):
    for j in range(i+1, 20):
        test_g.add_edge(i, j)
        test_g.add_edge(j, i)

In [34]:
def initiate_f_x(road_network, avs, nyc = False):
    previous_x = {}
    previous_f = {}
    for a in avs:
        for i,j in road_network.edges():
            previous_x[i,j,a.id] = 0
        for i in road_network.nodes():
            previous_f[i,a.id] = 0
        
        if nyc:
            previous_f[a.start_depot,a.id] += 1
            continue
        previous_f[a.id+5,a.id] += 1
        #print(a.id+5, a.id)
    return previous_f, previous_x

In [35]:
previous_f_nyc,_ = initiate_f_x(nyc_graph, vehicles, nyc = True)

In [36]:

previous_f, _ = initiate_f_x(test_g, vehicles)
#for i in previous_f.keys():
#    if previous_f[i] == 1:
#        print(previous_f[i],i)
previous_f

KeyError: (20, 15)

In [37]:
test_requests = []
for i in range(5):
    test_requests += [
        Request(id = i, people = 2, start=i+7, end=i+8 )
    ]


In [38]:
for r in test_requests:
    print(r.start, r.end)

7 8
8 9
9 10
10 11
11 12


In [43]:
time_list = convert_to_time_list(requests[:10], 10)
time_list

{0: [<__main__.Request at 0x7f7e90a2d0a0>],
 1: [<__main__.Request at 0x7f7e90a2daf0>],
 2: [<__main__.Request at 0x7f7e90a2dd30>],
 3: [<__main__.Request at 0x7f7e90a2dbe0>],
 4: [<__main__.Request at 0x7f7e90a47ee0>],
 5: [<__main__.Request at 0x7f7e90a2da30>],
 6: [<__main__.Request at 0x7f7e90a2dac0>],
 7: [<__main__.Request at 0x7f7e90a2d9d0>],
 8: [<__main__.Request at 0x7f7e90a2dd90>],
 9: [<__main__.Request at 0x7f7e90a2ddf0>]}

In [39]:
def get_se_list(requests):
    starts = []
    ends = []
    for r in requests:
        starts += [r.start]
        ends += [r.end]
    return starts, ends

def convert_to_time_list(requests, N):
    # Calculate the number of requests per interval
    requests_per_interval = len(requests) // N
    # Initialize a dictionary to store requests for each time interval
    time_requests = {t: [] for t in range(N)}
    
    # Iterate over each time interval
    for t in range(N):
        # Calculate the start and end indices for the requests in this interval
        start_index = t * requests_per_interval
        end_index = (t + 1) * requests_per_interval
        
        # Add requests to the current time interval
        time_requests[t] = requests[start_index:end_index]
    
    return time_requests

In [44]:
def set_up_opt_problem(road_network, vehicles, time_delta, previous_f,requests,d,previous_x={}, N = 10,
                       l_ij = 60,v_th = 100,v_max = 150,eps = 0.5, requests_per_interval=True
                       ):
    
    problem = pulp.LpProblem("MPC_ATOD", pulp.LpMinimize)

    b = (l_ij - eps) / (v_max - v_th) 

    V = {} 
    v = {}
    w = {}
    x = {}
    s = {}
    f = {}
    more_l = {}
    M = 10000000000


    for t in range(N):
        for i,j in road_network.edges():
            V[i,j, t] = pulp.LpVariable(f"V_{i},{j}-{t}", cat = "Integer", lowBound=0, upBound=v_max) 
            s[i,j, t] = pulp.pulp.LpVariable(f"s_{i},{j}-{t}", upBound=60, lowBound=eps) 
            more_l[i,j, t] = pulp.pulp.LpVariable(f"more_l_{i},{j}-{t}", cat = "Binary") 


            problem += s[(i, j,t)] >= l_ij- M * (1-more_l[(i, j,t)]) 
            problem += s[(i, j,t)] <= l_ij -  (l_ij - eps) / (v_th - v_max) *  (v_th- V[i,j,t]) #- M * more_l[(i, j)]
            problem += s[(i, j,t)] >= l_ij -  (l_ij - eps) / (v_th - v_max) *  (v_th- V[i,j,t]) - M * more_l[(i, j,t)]
            
            for a in vehicles:
                w[i,j,a.id,t] = pulp.pulp.LpVariable(f"w_{i},{j}-{a.id}-{t}", cat='Binary') 
                v[i,j,a.id,t] = pulp.pulp.LpVariable(f"v_{i},{j}-{a.id}-{t}", cat='Binary') 
                x[i,j,a.id,t] = pulp.pulp.LpVariable(f"x_{i},{j}-{a.id}-{t}",lowBound=0) 
                if t ==0:
                    problem += x[i,j,a.id,0] == 0
                    problem += x[i,j,a.id,0] == 0
                    problem += v[i,j,a.id,0] == 0
                    problem += w[i,j,a.id,0] == 0
                    continue
                problem+=  x[i,j,a.id,t] <= M*(w[i,j,a.id,t-1]+ v[i,j,a.id,t-1])
                problem += x[i,j,a.id,t] >= x[i,j,a.id,t-1]*time_delta + s[(i, j,t-1)] - M * (1-pulp.lpSum((w[i,j,a.id,t-1], v[i,j,a.id,t-1])))
                problem += x[i,j,a.id,t] <= x[i,j,a.id,t-1]*time_delta + s[(i, j,t-1)] 



            problem += V[i,j,t] == pulp.lpSum(w[i,j,a.id,t]+ v[i,j,a.id,t] for a in vehicles)
            
    

    
    
    arrived = {}
    for (i, j) in road_network.edges():
        for a in vehicles:
            for t in range(0, N):
                arrived[i,j,a.id,t] = pulp.pulp.LpVariable(f"arrived_{i},{j}-{a.id}-{t}", cat='Binary')
                if t ==0:
                    problem += arrived[i,j,a.id,t] == 0
                    continue
                problem+= x[i,j,a.id,t] - arrived[i,j,a.id,t]*d >=0
                problem+= x[i,j,a.id,t] -d+1- arrived[i,j,a.id,t]*M<=0

                problem+=arrived[i,j,a.id,t] >= arrived[i,j,a.id,t-1] -1
                problem+=arrived[i,j,a.id,t] <= 1-arrived[i,j,a.id,t-1]



    #for (i, j) in road_network.edges():
    #    for a in vehicles:
    #        for t in range(1, N):
    #            problem += w[i,j,a.id,t-1] >=  w[i,j,a.id,t] 
    #            problem += w[i,j,a.id,t-1] - x[i,j,a.id,t-1] *1/d <=  w[i,j,a.id,t] 
    #            problem += v[i,j,a.id,t-1] <=  v[i,j,a.id,t] 
    #            problem += v[i,j,a.id,t-1] - x[i,j,a.id,t-1]*1/d <=  v[i,j,a.id,t] 
                

    for (i, j) in road_network.edges():
            for a in vehicles:
                for t in range(1, N):
                    problem += w[i,j,a.id,t] <= 1 - v[i,j,a.id,t]
                    problem += w[i,j,a.id,t] <= 1 - v[i,j,a.id,t-1]
                    problem += w[i,j,a.id,t] <= 1 - arrived[i,j,a.id,t-1]
                    problem += w[i,j,a.id,t] >= v[i,j,a.id,t] + v[i,j,a.id,t-1] +arrived[i,j,a.id,t-1] -2
#
                    problem += v[i,j,a.id,t] <= 1 - w[i,j,a.id,t]
                    problem += v[i,j,a.id,t] <= 1 - w[i,j,a.id,t-1]
                    problem += v[i,j,a.id,t] <= 1 - arrived[i,j,a.id,t-1]
                    problem += v[i,j,a.id,t] >= w[i,j,a.id,t] + w[i,j,a.id,t] +arrived[i,j,a.id,t-1] -2

    
    departed = {}
    for t in range(1,N+1):
        for (i, j) in road_network.edges():
            for a in vehicles:
                departed[i,j,a.id,t-1] = pulp.pulp.LpVariable(f"departed_{i},{j}-{a.id}-{t-1}", cat='Binary')
                if t==1:
                    problem +=departed[i,j,a.id,t-1] == (v[i,j,a.id,t-1]+w[i,j,a.id,t-1])
                else:
                    problem +=departed[i,j,a.id,t-1] <= (v[i,j,a.id,t-1]+w[i,j,a.id,t-1]) 
                    problem +=departed[i,j,a.id,t-1] <= (1-(v[i,j,a.id,t-2]+w[i,j,a.id,t-2]) )
                    problem +=departed[i,j,a.id,t-1] >= (v[i,j,a.id,t-1]+w[i,j,a.id,t-1]) +(1-(v[i,j,a.id,t-2]+w[i,j,a.id,t-2]))-1
                    #problem +=departed[i,j,a.id,t-1] <= (v[i,j,a.id,t-1]+w[i,j,a.id,t-1]) - (v[i,j,a.id,t-2]+w[i,j,a.id,t-2])+1
                    
                    #problem +=departed[i,j,a.id,t-1] == arrived[i,j,a.id,t-2]

    
    print(departed)


    f = {}
    for t in range(N+1):
        for a in vehicles:
            for i in road_network.nodes(): 
                f[i,a.id, t] = pulp.pulp.LpVariable(f"f{i}-{a.id}-{t}", cat='Binary') 
                if t ==0:
                    problem+= f[i,a.id,t] == previous_f[i,a.id]
                    continue   
                #if t < N+1:
                problem+= f[i,a.id,t] == f[i,a.id,t-1] \
                                                - pulp.lpSum((departed[i,j,a.id,t-1]) for j in road_network.nodes() if (i,j) in road_network.edges())\
                                                + pulp.lpSum((arrived[j,i,a.id,t-1]) for j in road_network.nodes() if (j,i) in road_network.edges())
                    
                
                #to stop the vehicles, one can impose this constraint
                #problem+= f[i,a.id,t] == pulp.lpSum((arrived[j,i,a.id,t-1]) for j in road_network.nodes() if (j,i) in road_network.edges())


    
    for t in range(1,N):
         for a in vehicles:
                 problem +=pulp.lpSum(pulp.lpSum(f[i, a.id,t]for i in road_network.nodes())\
                                +pulp.lpSum(w[i,j, a.id,t-1]for i,j in road_network.edges())\
                                +pulp.lpSum(v[i,j, a.id,t-1]for i,j in road_network.edges())\
                                #-pulp.lpSum((arrived[i,j,a.id,t-1]) for i,j in road_network.edges())\
                                #pulp.lpSum((departed[i,j,a.id,t-1]) for i,j in road_network.edges())
                             )==1 

                      
    op = {}
    if requests_per_interval:
        time_list = convert_to_time_list(requests, N)

        for r in requests:
            if (r.start, r.end,0) not in op.keys():
                op[r.start, r.end, 0] =pulp.LpVariable(f"op{r.start}_{r.end}_{0}", cat="Integer", lowBound=0)
            if r in time_list[0]:
                problem += op[r.start, r.end, 0] == r.people_ 
            else:
                problem += op[r.start, r.end, 0] ==0


        for t in range(1,N+1):
            for r in requests:
                if (r.start, r.end,t) not in op.keys():
                    op[r.start, r.end, t] =pulp.LpVariable(f"op{r.start}_{r.end}_{t}", cat="Integer", lowBound=0)
                if t <N:
                    to_add = r.people_ if r in time_list[t] else 0 
                else:
                    to_add=0
                problem += op[r.start, r.end, t] == op[r.start, r.end, t - 1] +to_add - pulp.lpSum(a.pcapacity_ *v[r.start, r.end, a.id, t - 1] for a in vehicles)

    else:
        
        for r in requests:
            for t in range(N + 1):
                if (r.start, r.end, t) not in op.keys():
                    op[r.start, r.end, t] = pulp.LpVariable(f"op{r.start}_{r.end}_{t}", cat="Integer", lowBound=0)
                if t == 0:
                    problem += op[r.start, r.end, t] == r.people_  # Setting initial value for t=0
                else:
                    problem += op[r.start, r.end, t] >= 0  # Non-negativity constraint

                if t > 0:
                    problem += op[r.start, r.end, t] == op[r.start, r.end, t - 1] - pulp.lpSum(a.pcapacity_ *v[r.start, r.end, a.id, t - 1] for a in vehicles)
                    #pulp.lpSum(a.pcapacity_ * v[r.start, r.end, a.id, t - 1] for a in vehicles)

    

    #for r in requests:
    #    for t in range(N):
    #        problem += op[r.start, r.end, t] >= pulp.lpSum(v[r.start, r.end, a.id, t] for a in vehicles)


    problem += pulp.lpSum(op[i,j,t]  for i,j in road_network.edges() if (i,j,N) in op.keys() for t in range(N)) #- pulp.lpSum(f[r.end,a.id,N] for a in vehicles for r in requests)
    #problem += pulp.lpSum(op[i,j,N]  for i,j in road_network.edges() if (i,j,N) in op.keys() for t in range(N)) 
    #problem += -pulp.lpSum(v[r.start, r.end, a.id, t] for a in vehicles for r in requests for t in range(N))
    #problem += -pulp.lpSum(f[r.end,a.id,N] for a in vehicles for r in requests)
    #problem+=-pulp.lpSum(departed[i,j,a.id,t] for i,j in road_network.edges() for a in vehicles for t in range(1,N))
                
    #problem += -pulp.lpSum(v[i,j,a.id,t] for i,j in road_network.edges() for t in range(N) for a in vehicles )
    #problem += pulp.lpSum(w[i,j,a.id,t] for i,j in road_network.edges() for t in range(N) for a in vehicles )
    #problem += pulp.lpSum(f[i,a.id,N] for i in road_network.nodes() for a in vehicles if previous_f[i,a.id]==0)\
               #+pulp.lpSum(departed[i,j,a.id,t] for i,j in road_network.edges() for a in vehicles for t in range(N))

    return problem, V, v,w,s,f, x, departed, arrived,op
        


In [64]:
problem, V, v,w,s,f, x, departed, arrived,op = set_up_opt_problem(nyc_graph, vehicles,requests=requests[:100],
                                                               N = 10, time_delta=0.5, previous_f = previous_f_nyc, d = 120, requests_per_interval = True)

In [ ]:
solver = pulp.GUROBI_CMD(msg=1, timeLimit=600)

In [ ]:
solution = problem.solve(solver = solver)

Set parameter Username
Set parameter TimeLimit to value 600
Set parameter LogFile to value "gurobi.log"
Using license file /Users/brodie/gurobi.lic
Academic license - for non-commercial use only - expires 2025-02-08

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/78968ba3594b4c35819cb86526e49f7c-pulp.lp
Reading time = 14.19 seconds
OBJ: 6100530 rows, 1922040 columns, 18309270 nonzeros

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6100530 rows, 1922040 columns and 18309270 nonzeros
Model fingerprint: 0x56c23d9b
Variable types: 370230 continuous, 1551810 integer (1534070 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-01, 2e+02]
  RHS range        [1

In [48]:
graph = nyc_graph
t=3

In [49]:
for a in vehicles:
    for i,j in graph.edges():
        if pulp.value(v[i,j,a.id,t]) != None and pulp.value(v[i,j,a.id,t])>0.5:
            print(i,j,pulp.value(v[i,j,a.id,t]), a.id)

42442584 42442569 1.0 0
561035361 561035370 1.0 3
42436492 42450426.0 1.0 18
9166033061 42436492.0 1.0 19


In [50]:
for a in vehicles:
    for i,j in graph.edges():
        if pulp.value(w[i,j,a.id,t]) != None and pulp.value(w[i,j,a.id,t])>0.5:
            print(i,j,pulp.value(w[i,j,a.id,t]), a.id)

42442584 42432929 1.0 1
42432963 42432985 1.0 2
9166033061 42430828.0 1.0 4
9166033061 42430828.0 1.0 5
42445489 42428436 1.0 7
1061531654 1061531807 1.0 9
1061531654 1061531807 1.0 10
1061531654 1061531807 1.0 11
4597668041 42428473 1.0 15
42436492 42445357.0 1.0 17


In [51]:
for i,j in graph.edges():
    if pulp.value(V[i,j,t]) != None and pulp.value(V[i,j,t])>0.5:
        print(i,j,pulp.value(V[i,j,t]))

42432963 42432985 1.0
4597668041 42428473 1.0
9166033061 42436492.0 1.0
9166033061 42430828.0 2.0
1061531654 1061531807 3.0
42442584 42432929 1.0
42442584 42442569 1.0
42445489 42428436 1.0
42436492 42445357.0 1.0
42436492 42450426.0 1.0
561035361 561035370 1.0


In [52]:
for i,j in graph.edges():
    break
    if pulp.value(s[i,j,t]) != None and pulp.value(s[i,j,t])>=0:
        print(i,j,pulp.value(s[i,j,t]))

In [53]:
for a in vehicles:
    for i,j in graph.edges():
        if pulp.value(x[i,j,a.id,t]) != None and pulp.value(x[i,j,a.id,t])>0:
            print(i,j,pulp.value(x[i,j,a.id,t]), a.id)

42432929 42442584 60.0 0
42432929 42442584 60.0 1
42434262 561035361 60.0 3
42430041 9166033061.0 60.0 4
42430041 9166033061.0 60.0 5
42432165 42445489 60.0 7
6223571523 6223571521 60.0 8
3579432156 1061531654 60.0 9
3579432156 1061531654 60.0 10
3579432156 1061531654 60.0 11
42448693 4597668041 60.0 15
42447020 42436492.0 60.0 17
42447020 42436492.0 60.0 18
42447020 9166033061.0 60.0 19


In [54]:
for a in vehicles:
    for i in graph.nodes():
        if pulp.value(f[i,a.id,t]) != None and pulp.value(f[i,a.id,t])>0.5:
            print(i,pulp.value(f[i,a.id,t]), a.id)

42432963 1.0 2
42430044 1.0 6
3786901738 1.0 12
3786901738 1.0 13
3786901738 1.0 14
42447020 1.0 16


In [55]:
for a in vehicles:
    for i in graph.nodes():
        if pulp.value(f[i,a.id,t+1]) != None and pulp.value(f[i,a.id,t+1])>0.5:
            print(i,pulp.value(f[i,a.id,t+1]), a.id)

42430044 1.0 6
6223571521 1.0 8
3786901738 1.0 12
3786901738 1.0 13
3786901738 1.0 14
42447020 1.0 16


In [56]:
for a in vehicles:
    for i,j in graph.edges():
        if pulp.value(departed[i,j,a.id,t]) != None and pulp.value(departed[i,j,a.id,t])>0:
            print(i,j,pulp.value(departed[i,j,a.id,t]), a.id)

42442584 42442569 1.0 0
42442584 42432929 1.0 1
42432963 42432985 1.0 2
561035361 561035370 1.0 3
9166033061 42430828.0 1.0 4
9166033061 42430828.0 1.0 5
42445489 42428436 1.0 7
1061531654 1061531807 1.0 9
1061531654 1061531807 1.0 10
1061531654 1061531807 1.0 11
4597668041 42428473 1.0 15
42436492 42445357.0 1.0 17
42436492 42450426.0 1.0 18
9166033061 42436492.0 1.0 19


In [57]:
for a in vehicles:
    for i,j in graph.edges():
        if pulp.value(arrived[i,j,a.id,t]) != None and pulp.value(arrived[i,j,a.id,t])>0:
            print(i,j,pulp.value(arrived[i,j,a.id,t]), a.id)

42432929 42442584 1.0 0
42432929 42442584 1.0 1
42434262 561035361 1.0 3
42430041 9166033061.0 1.0 4
42430041 9166033061.0 1.0 5
42432165 42445489 1.0 7
6223571523 6223571521 1.0 8
3579432156 1061531654 1.0 9
3579432156 1061531654 1.0 10
3579432156 1061531654 1.0 11
42448693 4597668041 1.0 15
42447020 42436492.0 1.0 17
42447020 42436492.0 1.0 18
42447020 9166033061.0 1.0 19


In [58]:
for info in op.keys():
    print(info,pulp.value(op[info]))

(42430828.0, 42429552.0, 0) 2.0
(42447020.0, 9166033061.0, 0) 0.0
(9166033061.0, 42436492.0, 0) 0.0
(42450426.0, 42451593.0, 0) 0.0
(42430828.0, 9166033061.0, 0) 0.0
(42445357.0, 42442463.0, 0) 0.0
(42447020.0, 42436439.0, 0) 0.0
(42442889.0, 42436492.0, 0) 0.0
(9166033061.0, 42432706.0, 0) 0.0
(9166033061.0, 42429342.0, 0) 0.0
(42430828.0, 42429552.0, 1) 2.0
(42447020.0, 9166033061.0, 1) 2.0
(9166033061.0, 42436492.0, 1) 0.0
(42450426.0, 42451593.0, 1) 0.0
(42430828.0, 9166033061.0, 1) 0.0
(42445357.0, 42442463.0, 1) 0.0
(42447020.0, 42436439.0, 1) 0.0
(42442889.0, 42436492.0, 1) 0.0
(9166033061.0, 42432706.0, 1) 0.0
(9166033061.0, 42429342.0, 1) 0.0
(42430828.0, 42429552.0, 2) 2.0
(42447020.0, 9166033061.0, 2) 2.0
(9166033061.0, 42436492.0, 2) 2.0
(42450426.0, 42451593.0, 2) 0.0
(42430828.0, 9166033061.0, 2) 0.0
(42445357.0, 42442463.0, 2) 0.0
(42447020.0, 42436439.0, 2) 0.0
(42442889.0, 42436492.0, 2) 0.0
(9166033061.0, 42432706.0, 2) 0.0
(9166033061.0, 42429342.0, 2) 0.0
(42430828.

In [59]:
def save_info(graph,v,w,V, op,requests, N,folder_path,name_road = 'road_info.csv',  name_edge_info = "edge_info.csv", name_request_info = "requests_info.csv"):
    for t in range(N):
        road_info = {}
        for i, j in graph.edges():
            road_info[f"{i}-{j}"] = pulp.value(V[i,j,t])
        df = pd.DataFrame.from_dict(road_info, orient='index', columns=["use"]).reset_index().rename(columns={'index': 'edge'})
        df.to_csv(os.path.join(folder_path, f"{t:04d}_{name_road}"))


    for t in range(N):
        road_info = {}
        for i, j in graph.edges():
            edge_used = []
            for a in vehicles:
                edge_used.append(pulp.value(v[i,j,a.id,t]))
                
            road_info[f"{i}-{j}"] = edge_used

        df = pd.DataFrame.from_dict(road_info, orient='index', columns=[a.id for a in vehicles]).reset_index().rename(columns={'index': 'edge'})
        df.to_csv(os.path.join(folder_path, f"{t:04d}_vehicles_{name_road}"))

    for t in range(N):
        road_info = {}
        for i, j in graph.edges():
            edge_used = []
            for a in vehicles:
                edge_used.append(pulp.value(w[i,j,a.id,t]))
            road_info[f"{i}-{j}"] = edge_used
        df = pd.DataFrame.from_dict(road_info, orient='index', columns=[a.id for a in vehicles]).reset_index().rename(columns={'index': 'edge'})
        df.to_csv(os.path.join(folder_path, f"{t:04d}_vehicles_rebalancing_{name_road}"))


    for t in range(N):
        requests_info = {}
        for r in requests:
            requests_info[r.id] = [
                                    pulp.value(op[r.start,r.end, t]),
                                    r.start,
                                    r.end,
                                    [ a.id for a in vehicles if v[r.start,r.end,a.id,t]==1 ]
                                    ]

        
        pd.DataFrame.from_dict(requests_info, orient='index', columns=[
                                                        "quantity",
                                                        "start",
                                                        "end",
                                                        "which_vehicle"
                                                    ]).reset_index().rename(columns={'index': 'id'}).to_csv(os.path.join(folder_path, f"{t:04d}_{name_request_info}"))


        

    
    

In [60]:
folder_path = check_and_create_folder("mpc_simulation3", name_fold = "simu")

In [62]:
save_info(graph = nyc_graph,
              v = v,
              w = w,
              V = V, 
              op = op,
              requests=requests[:10],
              N = 10,
              folder_path = folder_path)
    